In [7]:
from datetime import datetime


import telebot


from fast_predictions import master_game

def formatted_time():
    return f'{datetime.now().day}.{datetime.now().month}.{datetime.now().year} ' + \
           f'{datetime.now().hour}:{datetime.now().minute}:{datetime.now().second}' + ' - '

bot_token = '5038632116:AAHhG5o_rnNpiWBjyF09Zyoh47x5oMLTo8s'
parameters_dictionary = {}
last_message = {}

bot = telebot.TeleBot(token=bot_token)
print(formatted_time() + 'бот запущен;')


# action_keyboard = telebot.types.ReplyKeyboardMarkup()
# action_keyboard.row('Запрос таблицы')
# action_keyboard.row('Отправить данные')
# action_keyboard.row('Остановить бота')

@bot.message_handler(commands=['start', 'help', 'send_info', 'get_prediction', 'our_results', 'stop'])
def start_message(message):
    last_message[message.from_user.id] = message.text
    if message.text == '/start':
        bot.send_message(
            message.chat.id,
            'Приветствую, это бот для прогноза проницаемости и литологии керна по данным компании «Газпром нефть»‎. ' +
            'Для отправки данных воспользуйтесь командой «/help»‎ ' +
            'запросите таблицу! Выберите требуемое действие:',
        )
    elif message.text == '/help':
        bot.send_message(
            message.chat.id,
            '/our_results - просмотреть наши предсказания \n' +
            '/send_info - включить режим прогноза\n' +
            '/get_prediction - получить прогноз по введённым данным'
        )
    elif message.text == '/stop' and message.from_user.id == 395411183:
        bot.send_message(message.chat.id, 'Бот остановлен.')
        print(formatted_time() + 'бот отключается;')
        bot.stop_polling()
    elif message.text == '/our_results':
        bot.send_message(message.chat.id,
                         'Бот использует алгоритмы машинного обучения, и в процессе этого ' +
                         'обучения мы пытались достичь наименьшей погрешности в прогнозе ' +
                         'для тестовых скважин. Используя последнюю конфигурацию модели, вы можете ' +
                         'увидеть наш прогноз (зелёный - ' +
                         'прогноз без коэффициента, оранжевый - прогноз с коэффициентом, синий ' +
                         '- реальные данные):'
                         )
        bot.send_photo(message.chat.id, photo='https://drive.google.com/open?id=1cOZPlYZLaejUoBJtTBZneU4OyNnfUdAs')
    elif message.text == '/send_info':
        bot.send_message(message.chat.id,
                         'Введите следующие целочисленные показатели в указанном порядке (один параметр -' +
                         ' одна строка, тольцо число), а после воспользуйтесь командой «/get_prediction»:\n' +
                         '1) Пористость\n' +
                         '2) Водосодержание(от 8300000 до 8900000);\n' +
                         '3) Гамма каротаж (от 20130000 до 20150000);\n'
                         )
    elif message.text == '/get_prediction':
        bot.send_message(message.chat.id,
                         'Примерный показатель для этих данных - ' +
                         f'{master_game(parameters_dictionary[message.from_user.id])} ')
    print(f'{formatted_time()}пользователь с ID {message.from_user.id} запросил команду «{message.text}»‎;')



@bot.message_handler(content_types=['text'])
def send_text(message):
    if (len(message.text.split('\n')) == 5) and (last_message[message.from_user.id] == '/send_info'):
        parameters_dictionary[message.from_user.id] = message.text.split('\n')
        for element in parameters_dictionary[message.from_user.id]:
            element = float(element)


bot.polling()
print(formatted_time() + 'бот отключен;')
quit(0)


29.12.2021 17:35:32 - бот запущен;
29.12.2021 17:35:36 - пользователь с ID 1043311118 запросил команду «/start»‎;
29.12.2021 17:59:5 - бот отключен;


In [2]:
df = pd.read_csv(r"Itog_data.csv")
df.drop(labels=['Unnamed: 16'], axis=1, inplace=True)
df.drop(labels=['DS'], axis=1, inplace=True) # решили не делить датасет, а просто удалить колонки
df.drop(labels=['RT'], axis=1, inplace=True)
df.drop(labels=['PEF'], axis=1, inplace=True)
df.drop(labels=['THOR'], axis=1, inplace=True)
df.drop(labels=['WELL'], axis=1, inplace=True)
df.drop(labels=['GK'], axis=1, inplace=True)
df.drop(labels=['URAN'], axis=1, inplace=True)
df.drop(labels=['POTA'], axis=1, inplace=True)
df.drop(labels=['MRP'], axis=1, inplace=True)
df.drop(labels=['DTS'], axis=1, inplace=True)
df.drop(labels=['DTP'], axis=1, inplace=True)




perm = np.log(df['PERM'])
features = df.drop('PERM', axis = 1)

x_train, x_test, y_train, y_test = train_test_split(features,perm, test_size = 0.3, random_state = 10)

cbr = CatBoostRegressor(iterations=1000, learning_rate=1, depth=None,)
cbr.fit(x_train, y_train)
y_pred = cbr.predict(x_test)

perm = np.log(df['PERM'])
features = df.drop('PERM', axis = 1)

x_train, x_test, y_train, y_test = train_test_split(features,perm, test_size = 0.2, random_state = 10)

reqr = CatBoostRegressor()
reqr.fit(x_train, y_train)

print('точность прогноза для тренировочных данных = ', reqr.score(x_train, y_train))
print('точность прогноза для тестовых данных  = ', reqr.score(x_test, y_test))


a = y_pred
data_3 = pd.DataFrame({'Проницаемость прогноз':a})

data_3

# сохранение моделей

master_model_path = 'Master Model.sav'
pickle.dump(master_lasso_model, open(master_model_path, 'wb'))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle


def our_tests():
    """

    :return: list
    """
    our_results = [2673096, 2777992, 2864085, 2953692, 2964733, 2586238]
    our_results_coefficient = [2673096, 2777992, 2864085, 2953692, 2964733, 2586238]
    true_results = [2712515, 2832963, 2929716, 2993037, 2922570, 2622023]
    mistake = np.sqrt(mean_squared_error(true_results, our_results))
    coefficient = 1.01
    for checker in range(6):
        our_results_coefficient[checker] = our_results_coefficient[checker] * coefficient
    mistake_coefficient = np.sqrt(mean_squared_error(true_results, our_results_coefficient))
    return [[our_results, mistake], [our_results_coefficient, mistake_coefficient], true_results]

def master_game(parameters):
    """

    :param parameters:
    :type parameters: list
    :return: int
    """
    master_model = pickle.load(open('Master Model.sav',
                                    mode='rb'))
    parameters[2] = int(parameters[2]) / 10000
    new_parameters = pd.DataFrame([parameters, parameters], dtype=float)
    result = master_model.predict(new_parameters)
    return int(result[0])